In [1]:
import mxnet as mxt
import gluonts
import numpy as np
import pandas as pd
import os
import json

In [2]:
!mkdir datasets

mkdir: datasets: File exists


In [3]:
!ls -la

total 194592
drwxr-xr-x   6 sharabh_shukla  staff       192 Nov 11 03:24 .
drwxr-xr-x  22 sharabh_shukla  staff       704 Nov 11 03:24 ..
drwxr-xr-x   3 sharabh_shukla  staff        96 Nov 11 03:24 .ipynb_checkpoints
-rw-r--r--   1 sharabh_shukla  staff  99603157 May 27 13:57 EMHIRESPV_TSh_CF_Country_19862015.xlsx
-rw-r--r--   1 sharabh_shukla  staff     22417 Nov 11 03:24 Solar_forecasting.ipynb
drwxr-xr-x   2 sharabh_shukla  staff        64 Nov 11 00:54 datasets


In [4]:
!cd datasets
!wget https://setis.ec.europa.eu/sites/default/files/EMHIRES_DATA/Solar/EMHIRESPV_country_level.zip

--2019-11-11 21:44:39--  https://setis.ec.europa.eu/sites/default/files/EMHIRES_DATA/Solar/EMHIRESPV_country_level.zip
Resolving setis.ec.europa.eu (setis.ec.europa.eu)... 139.191.207.52
Connecting to setis.ec.europa.eu (setis.ec.europa.eu)|139.191.207.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93401258 (89M) [application/zip]
Saving to: ‘EMHIRESPV_country_level.zip’

EMHIRESPV_country_l 100%[===================>]  89.07M  11.7MB/s    in 8.7s    

2019-11-11 21:44:48 (10.3 MB/s) - ‘EMHIRESPV_country_level.zip’ saved [93401258/93401258]



In [5]:
!unzip -o EMHIRESPV_country_level.zip

Archive:  EMHIRESPV_country_level.zip
  inflating: EMHIRESPV_TSh_CF_Country_19862015.xlsx  


In [6]:
!rm EMHIRESPV_country_level.zip

In [7]:
%%time
data_xlsx = pd.read_excel('EMHIRESPV_TSh_CF_Country_19862015.xlsx' )

CPU times: user 1min 39s, sys: 878 ms, total: 1min 40s
Wall time: 1min 40s


In [8]:
def process_time(df, freq='1H'):
    
    # Convert timestamp into a pandas datatime object
    #df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    #df = df.set_index('Timestamp')
    
    # Extract units of time from the timestamp
    df['min'] = df.index.minute
    df['hour'] = df.index.hour
    df['wday'] = df.index.dayofweek
    df['mday'] = df.index.day - 1
    df['yday'] = df.index.dayofyear - 1
    df['month'] = df.index.month - 1
    df['year'] = df.index.year
    
    # Create a time of day to represent hours and minutes
    df['time'] = df['hour'] + (df['min'] / 60)
    df = df.drop(columns=['hour', 'min'])
    
    # Cyclical variable transformations
    
    # wday has period of 6
    df['wday_sin'] = np.sin(2 * np.pi * df['wday'] / 6)
    df['wday_cos'] = np.cos(2 * np.pi * df['wday'] / 6)
    
    # yday has period of 365
    df['yday_sin'] = np.sin(2 * np.pi * df['yday'] / 364)
    df['yday_cos'] = np.cos(2 * np.pi * df['yday'] / 364)
    
    # month has period of 12
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    
    # time has period of 24
    df['time_sin'] = np.sin(2 * np.pi * df['time'] / 23)
    df['time_cos'] = np.cos(2 * np.pi * df['time'] / 23)
    
    df['past_hour_load_1'] = df['Generation'].shift(-1)
    df['past_hour_load_2'] = df['Generation'].shift(-2)
    df['past_hour_load_3'] = df['Generation'].shift(-3)
    df['past_hour_load_4'] = df['Generation'].shift(-4)
    df['past_hour_load_5'] = df['Generation'].shift(-5)
    df['past_hour_load_6'] = df['Generation'].shift(-6)
    df['past_hour_load_7'] = df['consumption_rate'].shift(-7)
    df['past_hour_load_8'] = df['consumption_rate'].shift(-8)
    df['past_hour_load_9'] = df['consumption_rate'].shift(-9)
    df['past_hour_load_10'] = df['consumption_rate'].shift(-10)
    df['past_hour_load_11'] = df['consumption_rate'].shift(-11)
    df['past_hour_load_12'] = df['consumption_rate'].shift(-12)
    
    # turn the index into a column
    #df = df.reset_index(level=0)
    
    return df
  
    
def is_df_missing_steps(df, freq='1H'):
    index_steps = df.index
    start = df.index[0]
    end = df.index[-1]
    dt_series = pd.date_range(start=start, end= end, freq = freq)
    return not dt_series.equals(index_steps)

def get_missing_steps(df, freq='1H'):
    index_steps = df.index
    start = df.index[0]
    end = df.index[-1]
    dt_series = pd.date_range(start=start, end= end, freq=freq)
    return dt_series.difference(index_steps)

In [9]:
id_vars = ['Time_step','Date','Year','Month','Day','Hour']
data_tall = pd.melt(data_xlsx, id_vars=id_vars, var_name='Country', value_name='Generation')

In [10]:
data_xlsx.set_index('Date', inplace=True)

In [11]:
data_tall.set_index('Date', inplace=True)
data_tall.head()

,Time_step,Year,Month,Day,Hour,Country,Generation
Date,,,,,,,
1986-01-01 00:00:00,1,1986,1,1,0,AL,0.0
1986-01-01 01:00:00,2,1986,1,1,1,AL,0.0
1986-01-01 02:00:00,3,1986,1,1,2,AL,0.0
1986-01-01 03:00:00,4,1986,1,1,3,AL,0.0
1986-01-01 04:00:00,5,1986,1,1,4,AL,0.0


In [12]:
is_df_missing_steps(data_xlsx)

False

In [13]:
get_missing_steps(data_xlsx)

DatetimeIndex([], dtype='datetime64[ns]', freq='H')

In [14]:
data_xlsx.isna().any(axis=1).sum()

0

In [15]:
data_xlsx.head()

,Time_step,Year,Month,Day,Hour,AL,AT,BA,BE,BG,...,NO,PL,PT,RO,RS,SI,SK,SE,XK,UK
Date,,,,,,,,,,,,,,,,,,,,,
1986-01-01 00:00:00,1,1986,1,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1986-01-01 01:00:00,2,1986,1,1,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1986-01-01 02:00:00,3,1986,1,1,2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1986-01-01 03:00:00,4,1986,1,1,3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1986-01-01 04:00:00,5,1986,1,1,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:


target_cols = list(set(data_xlsx.columns) - set(['Time_step','Year','Month','Day','Hour']))
target_cols

['ME',
 'XK',
 'EL',
 'FI',
 'CZ',
 'BG',
 'BE',
 'ES',
 'DE',
 'CH',
 'SK',
 'PT',
 'RS',
 'RO',
 'HR',
 'SE',
 'IT',
 'DK',
 'LT',
 'LU',
 'EE',
 'SI',
 'MK',
 'FR',
 'IE',
 'BA',
 'HU',
 'CY',
 'PL',
 'NO',
 'UK',
 'AT',
 'AL',
 'LV',
 'NL']

In [17]:
from gluonts.dataset.field_names import FieldName

In [18]:
from gluonts.distribution.distribution_output import DistributionOutput
from gluonts.distribution.gaussian import GaussianOutput

In [19]:
from gluonts.dataset.common import ListDataset

In [20]:
train_ds = ListDataset([{
    FieldName.TARGET: data_xlsx[:-365][target_cols].to_numpy(),
    FieldName.START: data_xlsx.index[0],
    FieldName.FEAT_DYNAMIC_REAL: data_xlsx[:-365][['Year', 'Month', 'Day', 'Hour']].to_numpy()
    
}],
        freq = '1H',
        one_dim_target=False)

In [21]:
data_xlsx[:-365][['Year', 'Month', 'Day', 'Hour']].to_numpy()

array([[1986,    1,    1,    0],
       [1986,    1,    1,    1],
       [1986,    1,    1,    2],
       ...,
       [2015,   12,   16,   16],
       [2015,   12,   16,   17],
       [2015,   12,   16,   18]])

In [22]:
from gluonts.model.deepar import DeepAREstimator
from gluonts.trainer import Trainer

INFO:root:Using CPU


In [23]:
estimator = DeepAREstimator(freq="1H", prediction_length=24,context_length = 4*7*24, distr_output=GaussianOutput(), trainer=Trainer(epochs=10), num_layers = 4)

INFO:root:Using CPU


In [ ]:
predictor = estimator.train(training_data=train_ds)

INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/50 [00:00<?, ?it/s]